In [7]:
import pandas as pd
import os
import numpy as np
import logging
import sys
import torch
import copy
import yaml
import random

from prediction_utils.pytorch_utils.metrics import (
    StandardEvaluator,
    FairOVAEvaluator,
    CalibrationEvaluator
)

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.set_style("ticks")

grp_label_dict = {1: 'Black women', 2: 'White women', 3: 'Black men', 4: 'White men'} 

args = {'cohort_path': '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts/cohort/all_cohorts.csv',
        'base_path': '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts',
       }

#calib_method = 'loess'
calib_method = 'logx_logreg'

def get_real_prevalence_tables(df):
    means = (df
     .filter(['model_type', 'model_id', 'model_type', 'group', 'calibration_density', 'pred_probs'])
     .groupby(['pred_probs', 'model_id','group'])
     .mean()
     .rename_axis(columns = {'calibration_density': 'prevalence_at_threshold'})
    )

    stds = (df
     .filter(['model_type', 'model_id', 'model_type', 'group', 'calibration_density', 'pred_probs'])
     .groupby(['pred_probs', 'model_id','group'])
     .std()
     .rename_axis(columns = {'calibration_density': 'prevalence_at_threshold'})
    )

    lower = (means-2*stds).applymap(lambda x: format(x, '.4f'))
    higher = (means+2*stds).applymap(lambda x: format(x, '.4f'))
    means1 = means.applymap(lambda x: format(x, '.4f'))
    table = means1 + ' (' + lower + ', ' + higher + ')'
    assert len(df.model_type.unique()) == 1, "multiple model types passed"
    
    table = table.assign(model_type = df.model_type.unique()[0])

    return table

b=[]
eqodds_threshold = 0.1
for experiment in ['original_pce', 'apr14_erm', 'apr14_erm_recalib', 'apr14_mmd']: # 'apr14_mmd', 'apr14_thr',

    aggregate_path = os.path.join(args['base_path'], 'experiments', 
                                      experiment, 'performance',
                                      'all', 'calibration', calib_method)
    thr_calibs = pd.read_csv(os.path.join(aggregate_path, 'calibration_sensitivity_thresholds_raw.csv'))
    
    if experiment in ['apr14_mmd', 'apr14_thr']:
        thr_calibs = thr_calibs.query('model_id == @eqodds_threshold')
    
    a = thr_calibs.assign(group = lambda x: x.group.map(grp_label_dict))
    if 'model_id' not in a.columns:
        a = a.assign(model_id=0)

    table = get_real_prevalence_tables(a)
    b.append(table)

(pd
 .concat(b)
 .assign(model_type = lambda x: x.model_type.map({'original_pce': 'PCE',
                                                                     'erm': 'ERM',
                                                                     'recalib_erm': 'rERM',
                                                                     'eqodds_mmd': 'EqOdd'}
                                                                   ))
 .reset_index()
 .pivot(index=['pred_probs', 'group'], columns="model_type", values="calibration_density")
 .reindex(columns=['PCE', 'ERM', 'rERM', 'EqOdd'])
 .reset_index()
 .rename(columns = {'pred_probs' : 'decision threshold'})
 .set_index(['decision threshold', 'group'])
)

model_type                                          PCE  \
decision threshold group                                  
0.075              Black men    0.0440 (0.0233, 0.0647)   
                   Black women  0.0418 (0.0230, 0.0606)   
                   White men    0.0423 (0.0247, 0.0598)   
                   White women  0.0563 (0.0366, 0.0760)   
0.200              Black men    0.1206 (0.0738, 0.1673)   
                   Black women  0.1227 (0.0812, 0.1642)   
                   White men    0.1486 (0.1145, 0.1827)   
                   White women  0.2076 (0.1507, 0.2645)   

model_type                                          ERM  \
decision threshold group                                  
0.075              Black men    0.0719 (0.0443, 0.0995)   
                   Black women  0.0610 (0.0348, 0.0872)   
                   White men    0.0616 (0.0406, 0.0826)   
                   White women  0.0608 (0.0402, 0.0814)   
0.200              Black men    0.1669 (0.1061, 0.2277)   
                   Black women  0.1947 (0.1232, 0.2661)   
                   White men    0.2215 (0.1724, 0.2705)   
                   White women  0.2413 (0.1692, 0.3133)   

model_type                                         rERM  \
decision threshold group                                  
0.075              Black men    0.0808 (0.0510, 0.1106)   
                   Black women  0.0757 (0.0433, 0.1081)   
                   White men    0.0800 (0.0551, 0.1048)   
                   White women  0.0833 (0.0549, 0.1117)   
0.200              Black men    0.2414 (0.1417, 0.3410)   
                   Black women  0.2369 (0.1465, 0.3274)   
                   White men    0.1879 (0.1469, 0.2288)   
                   White women  0.2596 (0.1833, 0.3360)   

model_type                                        EqOdd  
decision threshold group                                 
0.075              Black men    0.0782 (0.0489, 0.1075)  
                   Black women  0.0513 (0.0264, 0.0763)  
                   White men    0.0700 (0.0472, 0.0927)  
                   White women  0.0529 (0.0333, 0.0726)  
0.200              Black men    0.1770 (0.1076, 0.2465)  
                   Black women  0.1827 (0.1152, 0.2502)  
                   White men    0.2391 (0.1845, 0.2937)  
                   White women  0.2227 (0.1542, 0.2913)

In [2]:
a = (pd
 .concat(b)
 .assign(model_type = lambda x: x.model_type.map({'original_pce': 'PCE',
                                                                     'erm': 'ERM',
                                                                     'recalib_erm': 'rERM',
                                                                     'eqodds_mmd': 'EqOdd'}
                                                                   ))
 .reset_index()
 .pivot(index=['pred_probs', 'group'], columns="model_type", values="calibration_density")
 .reindex(columns=['PCE', 'ERM', 'rERM', 'EqOdd'])
 .reset_index()
 .rename(columns = {'pred_probs' : 'decision threshold'})
 .set_index(['decision threshold', 'group'])
)

In [6]:
a.columns.name=''
a

PCE  \
decision threshold group                                  
0.075              Black men    0.0440 (0.0233, 0.0647)   
                   Black women  0.0418 (0.0230, 0.0606)   
                   White men    0.0423 (0.0247, 0.0598)   
                   White women  0.0563 (0.0366, 0.0760)   
0.200              Black men    0.1206 (0.0738, 0.1673)   
                   Black women  0.1227 (0.0812, 0.1642)   
                   White men    0.1486 (0.1145, 0.1827)   
                   White women  0.2076 (0.1507, 0.2645)   

                                                    ERM  \
decision threshold group                                  
0.075              Black men    0.0719 (0.0443, 0.0995)   
                   Black women  0.0610 (0.0348, 0.0872)   
                   White men    0.0616 (0.0406, 0.0826)   
                   White women  0.0608 (0.0402, 0.0814)   
0.200              Black men    0.1669 (0.1061, 0.2277)   
                   Black women  0.1947 (0.1232, 0.2661)   
                   White men    0.2215 (0.1724, 0.2705)   
                   White women  0.2413 (0.1692, 0.3133)   

                                                   rERM  \
decision threshold group                                  
0.075              Black men    0.0808 (0.0510, 0.1106)   
                   Black women  0.0757 (0.0433, 0.1081)   
                   White men    0.0800 (0.0551, 0.1048)   
                   White women  0.0833 (0.0549, 0.1117)   
0.200              Black men    0.2414 (0.1417, 0.3410)   
                   Black women  0.2369 (0.1465, 0.3274)   
                   White men    0.1879 (0.1469, 0.2288)   
                   White women  0.2596 (0.1833, 0.3360)   

                                                  EqOdd  
decision threshold group                                 
0.075              Black men    0.0733 (0.0448, 0.1018)  
                   Black women  0.0588 (0.0316, 0.0859)  
                   White men    0.0622 (0.0401, 0.0843)  
                   White women  0.0602 (0.0385, 0.0820)  
0.200              Black men    0.1697 (0.1061, 0.2332)  
                   Black women  0.1918 (0.1187, 0.2649)  
                   White men    0.2239 (0.1737, 0.2742)  
                   White women  0.2393 (0.1649, 0.3137)